<a href="https://colab.research.google.com/github/habiib1999/Speech-Emotion-Recognition/blob/main/emotionRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio
import os
import re

import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Audio
# from entropy import spectral_entropy
from keras import layers
from keras import models
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import itertools

# CLASS : READER

In [123]:
#READER CLASS
class Reader:
  def __init__(self, crema_path):
    self.crema_path = crema_path


  def readCrema(self):
    emotion_df = []

    for wav in os.listdir(crema_path):
      info = wav.partition(".wav")[0].split("_")
      if info[2] == 'SAD':
        emotion_df.append(("sad", crema_path + "/" + wav))
      elif info[2] == 'ANG':
        emotion_df.append(("angry", crema_path + "/" + wav))
      elif info[2] == 'DIS':
        emotion_df.append(("disgust", crema_path + "/" + wav))
      elif info[2] == 'FEA':
        emotion_df.append(("fear", crema_path + "/" + wav))
      elif info[2] == 'HAP':
        emotion_df.append(("happy", crema_path + "/" + wav))
      elif info[2] == 'NEU':
        emotion_df.append(("neutral", crema_path + "/" + wav))
      else:
        emotion_df.append(("unknown", crema_path + "/" + wav))


    Crema_df = pd.DataFrame.from_dict(emotion_df)
    Crema_df.rename(columns={1 : "Path", 0 : "Emotion"}, inplace=True)

    #print(Crema_df.head())
    return Crema_df

 

  def read(self):
    crema_dataset = self.readCrema()
    return crema_dataset
    

In [124]:
crema_path = "/content/drive/MyDrive/emotionDataset/Crema"
reader = Reader(crema_path)
crema_dataset = reader.read()
#from sklearn.utils import shuffle
#crema_dataset = shuffle(crema_dataset)
print(crema_dataset.head(10))

   Emotion                                               Path
0     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
1     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
2      sad  /content/drive/MyDrive/emotionDataset/Crema/10...
3  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
4  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
5    angry  /content/drive/MyDrive/emotionDataset/Crema/10...
6      sad  /content/drive/MyDrive/emotionDataset/Crema/10...
7  disgust  /content/drive/MyDrive/emotionDataset/Crema/10...
8     fear  /content/drive/MyDrive/emotionDataset/Crema/10...
9     fear  /content/drive/MyDrive/emotionDataset/Crema/10...


In [125]:
def energy(data, frame_length=2048, hop_length=512):
    en = np.array([np.sum(np.power(np.abs(data[hop:hop+frame_length]), 2)) for hop in range(0, data.shape[0], hop_length)])
    return en / frame_length
def mel_spc(data, sr, frame_length=2048, hop_length=512, flatten: bool = False):
    mel = librosa.feature.melspectrogram(y=data, sr=sr)
    return np.squeeze(mel.T) if not flatten else np.ravel(mel.T)

In [126]:
def extract_features(data, sr, frame_length=2048, hop_length=512):
    result = np.array([])

    return energy(data, frame_length, hop_length),mel_spc(data, sr, frame_length, hop_length)


def get_features(path, duration=2.5, offset=0.6):
    # duration and offset are used to take care of the no audio in start and the ending of each audio files as seen above.
    data, sample_rate = librosa.load(path, duration=duration, offset=offset)

    energy_feature,mel_spec_feature = extract_features(data, sample_rate)

    return np.array(energy_feature) , np.array(mel_spec_feature)

In [127]:
path = np.array(crema_dataset["Path"])[657]
data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
print(len(data))
print(data)
print(sampling_rate)
enrg = energy(data)
mel_spec = mel_spc(data, sampling_rate)
print("Energy shape: ", enrg.shape)
print("Energy: ", enrg)
print("MelSpectrogram shape: ", mel_spec.shape)
print("MelSpectrogram : ", mel_spec)

41215
[-0.00774202 -0.00921047 -0.00838774 ...  0.          0.
  0.        ]
22050
Energy shape:  (81,)
Energy:  [1.25416682e-05 9.00268205e-05 1.85334604e-04 3.20473308e-04
 4.43909288e-04 5.84825990e-04 6.93780254e-04 7.57424335e-04
 7.28843384e-04 5.22820163e-04 3.26876121e-04 2.08084093e-04
 3.37646983e-04 4.53232380e-04 6.37009856e-04 6.90810732e-04
 6.26968686e-04 6.22040359e-04 5.71376935e-04 5.53829479e-04
 5.40389272e-04 5.17104869e-04 4.33028355e-04 3.69260844e-04
 3.49385751e-04 6.80433295e-04 9.92599176e-04 1.06770382e-03
 1.07393204e-03 7.50555773e-04 3.96377407e-04 3.65105516e-04
 3.75498668e-04 3.81877937e-04 4.22568090e-04 4.03064187e-04
 3.02504777e-04 2.81216606e-04 3.60311329e-04 4.31530643e-04
 5.63004171e-04 7.37200084e-04 9.54805641e-04 1.11985242e-03
 1.10804976e-03 1.04999903e-03 7.55362853e-04 4.82189003e-04
 3.56042641e-04 1.70072773e-04 1.18846765e-04 8.40425055e-05
 5.14617059e-05 3.80887905e-05 3.09384013e-05 2.71017125e-05
 2.12698069e-05 1.17438049e-05 1.

In [ ]:
extracted_data_energy, extracted_data_mel_spec = [], []
extracted_data_energy_labels, extracted_data_mel_spec_labels = [], []
print("Feature processing...")
for path, emotion, ind in zip(crema_dataset.Path, crema_dataset.Emotion, range(crema_dataset.Path.shape[0])):
    energy_feature,mel_spec_feature = get_features(path)
    if ind % 100 == 0:
        print(f"{ind} samples has been processed...")
    extracted_data_energy.append(energy_feature)      
    extracted_data_energy_labels.append(emotion)
    extracted_data_mel_spec.append(mel_spec_feature)      
    extracted_data_mel_spec_labels.append(emotion)

print("Done.")

# Save ENERGY

In [160]:
'''
features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_features.csv"
extracted_df = pd.DataFrame(extracted_data_energy)
extracted_df["labels"] = extracted_data_energy_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()
'''
extracted_data_energy = np.array(extracted_data_energy)

features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_data_feature2.npy"
np.save(features_path, extracted_data_energy)

features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_labels_feature2.npy"
np.save(features_path, extracted_data_mel_spec_labels)
#np.savetxt(features_path, np.array(extracted_data_energy_labels), delimiter=",",fmt='%s')


# Save MEL SPECTROGRAM

In [161]:
'''
features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_features.csv"
extracted_df = pd.DataFrame(extracted_data_mel_spec)
extracted_df["labels"] = extracted_data_mel_spec_labels
extracted_df.to_csv(features_path, index=False)
extracted_df.head()
'''
extracted_data_mel_spec = np.array(extracted_data_mel_spec)

features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_data_features2.npy"
np.save(features_path, extracted_data_mel_spec)

features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_labels_feature2.npy"
np.save(features_path, extracted_data_energy_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


# Prepare ENERGY Feature

In [3]:
energy_data = np.load("/content/drive/MyDrive/emotionDataset/crema_energy_data_feature2.npy",allow_pickle=True) 
energy_labels = np.load( "/content/drive/MyDrive/emotionDataset/crema_energy_labels_feature2.npy",allow_pickle=True)
'''print(energy_data.shape)
print(energy_data)
print(energy_labels.shape)
print(energy_labels)'''


'print(energy_data.shape)\nprint(energy_data)\nprint(energy_labels.shape)\nprint(energy_labels)'

In [4]:
# FILL NaN
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out
energy_data = numpy_fillna(energy_data)


In [5]:
# Encode labels
lb = LabelEncoder()
energy_labels = np_utils.to_categorical(lb.fit_transform(energy_labels))
print(lb.classes_)
print(energy_labels)

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad']
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [6]:
# Split data
energy_train, energy_test, energy_label_train, energy_label_test = train_test_split(energy_data, energy_labels, test_size=0.3, random_state=1)

energy_train, energy_vald, energy_label_train, energy_label_vald = train_test_split(energy_train, energy_label_train, test_size=0.05, random_state=1) 

In [7]:
# Standardize data
print(energy_train)
scaler = StandardScaler()
energy_train = scaler.fit_transform(energy_train)
energy_test = scaler.transform(energy_test)
energy_vald = scaler.transform(energy_vald)


print(energy_train)
print(energy_train.shape)

[[0.007495223544538021 0.006455752998590469 0.006186658516526222 ... 0 0
  0]
 [3.3847958547994494e-05 3.0064849852351472e-05 0.0004387139924801886 ...
  0 0 0]
 [0.0005620798328891397 0.0010974573669955134 0.0018036244437098503 ... 0
  0 0]
 ...
 [3.178983024554327e-05 2.239243985968642e-05 2.1651685528922826e-05 ...
  0 0 0]
 [5.2499101002467796e-05 4.992265166947618e-05 4.332196112954989e-05 ...
  0 0 0]
 [0.00037560187047347426 0.0005270219990052283 0.0005730513948947191 ...
  0 0 0]]
[[ 0.11573338  0.05076604  0.02740831 ... -0.0272409  -0.02680934
  -0.02759062]
 [-0.28167873 -0.28374242 -0.26385252 ... -0.0272409  -0.02680934
  -0.02759062]
 [-0.25354374 -0.2281761  -0.19468954 ... -0.0272409  -0.02680934
  -0.02759062]
 ...
 [-0.28178835 -0.28414183 -0.28498597 ... -0.0272409  -0.02680934
  -0.02759062]
 [-0.28068532 -0.28270866 -0.28388789 ... -0.0272409  -0.02680934
  -0.02759062]
 [-0.26347603 -0.25787182 -0.25704535 ... -0.0272409  -0.02680934
  -0.02759062]]
(4948, 108)


# DISABLE

In [ ]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_energy_features.csv"
energy_labels_data= pd.read_csv(features_path)
energy_labels_data.head()
# FILL NaN
energy_labels_data = energy_labels_data.fillna(0)
energy_labels_data.shape

#Extract labels
energy_data = energy_labels_data.drop(labels="labels", axis=1)
energy_labels = energy_labels_data["labels"]

# Encode labels
lb = LabelEncoder()
energy_labels = np_utils.to_categorical(lb.fit_transform(energy_labels))
print(lb.classes_)
print(energy_labels)

# Split data
energy_train, energy_test, energy_label_train, energy_label_test = train_test_split(energy_data, energy_labels, test_size=0.3, random_state=1)

energy_train, energy_vald, energy_label_train, energy_label_vald = train_test_split(energy_train, energy_label_train, test_size=0.05, random_state=1) 


# Standardize data
scaler = StandardScaler()
energy_train = scaler.fit_transform(energy_train)
energy_test = scaler.transform(energy_test)
energy_vald = scaler.transform(energy_vald)


print(energy_train)
print(energy_train.shape)

# # Prepare MEL SPECTROGRAM Feature

In [8]:
mel_spec_data = np.load("/content/drive/MyDrive/emotionDataset/crema_mel_spec_data_features2.npy",allow_pickle=True) 
mel_spec_labels = np.load( "/content/drive/MyDrive/emotionDataset/crema_mel_spec_labels_feature2.npy",allow_pickle=True)

print(mel_spec_data[0].shape)

(67, 128)


In [9]:
# FILL NaN
'''
def pandas_fill(arr):   
    arr = pd.DataFrame(arr)
    arr = arr.fillna(0)
    return arr

print(max(map(len, mel_spec_data)))
#mel_spec_data = pandas_fill(mel_spec_data)
#mel_spec_data = mel_spec_data.to_numpy()
print(mel_spec_data[13].shape)
'''
mel_spec_data2 = []
def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out
for i in range(0, len(mel_spec_data)):
    mel_spec_data[i] = mel_spec_data[i].flatten()
print(mel_spec_data[0].shape)

mel_spec_data = numpy_fillna(mel_spec_data)
print(mel_spec_data[0].shape)

for i in range(0, len(mel_spec_data)):
    mel_spec_data2.append(np.reshape(mel_spec_data[i], (108,128) ))
mel_spec_data = mel_spec_data2
print(mel_spec_data[0].shape)


(8576,)
(13824,)
(108, 128)


In [10]:
# Encode labels
lb = LabelEncoder()
mel_spec_labels = np_utils.to_categorical(lb.fit_transform(mel_spec_labels))
print(lb.classes_)
print(mel_spec_labels)

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad']
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [11]:
# Split data
mel_spec_train, mel_spec_test, mel_spec_label_train, mel_spec_label_test = train_test_split(mel_spec_data, mel_spec_labels, test_size=0.3, random_state=1)

mel_spec_train, mel_spec_vald, mel_spec_label_train, mel_spec_label_vald = train_test_split(mel_spec_train, mel_spec_label_train, test_size=0.05, random_state=1) 

In [16]:
# Standardize data
print(mel_spec_data[7441].shape)
scaler = StandardScaler()
for i in range(0, len(mel_spec_train)):
    mel_spec_train[i] = scaler.fit_transform(mel_spec_train[i])
for i in range(0, len(mel_spec_test)):
    mel_spec_test[i] = scaler.transform(mel_spec_test[i])
for i in range(0, len(mel_spec_vald)):
    mel_spec_vald[i] = scaler.transform(mel_spec_vald[i])

(108, 128)


# Disable

In [ ]:
features_path = "/content/drive/MyDrive/emotionDataset/crema_mel_spec_features.csv"
mel_spec_labels_data = pd.read_csv(features_path)
print(mel_spec_labels_data.head())
print(mel_spec_labels_data.values)

#Fill NaN
mel_spec_labels_data = mel_spec_labels_data.fillna(0)
mel_spec_labels_data.shape

#Extract labels
mel_spec_data = mel_spec_labels_data.drop(labels="labels", axis=1)
mel_spec_labels = mel_spec_labels_data["labels"]

# Encode labels
lb = LabelEncoder()
mel_spec_labels = np_utils.to_categorical(lb.fit_transform(mel_spec_labels))
print(lb.classes_)
print(mel_spec_labels)

# Split data
mel_spec_train, mel_spec_test, mel_spec_label_train, mel_spec_label_test = train_test_split(mel_spec_data, mel_spec_labels, test_size=0.3, random_state=1)

mel_spec_train, mel_spec_vald, mel_spec_label_train, mel_spec_label_vald = train_test_split(mel_spec_train, mel_spec_label_train, test_size=0.05, random_state=1) 
print(mel_spec_train)
'''
mel_spec_train.columns = ['spect']
mel_spec_train = mel_spec_train['spect'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
'''
'''
from ast import literal_eval
mel_spec_train= np.array(literal_eval(mel_spec_train))
'''

for i in range(0, len(mel_spec_train)-1):
  mel_spec_train["0"] = np.array(mel_spec_train.iloc[i]["0"])
  continue


print(mel_spec_train)
print(type(mel_spec_train))
# Standardize data
scaler = StandardScaler()
mel_spec_train = scaler.fit_transform(mel_spec_train)
mel_spec_test = scaler.transform(mel_spec_test)
mel_spec_vald = scaler.transform(mel_spec_vald)


print(mel_spec_train)
print(type(mel_spec_train))